# 3. German dataset (Scarlett & Lily)

In [ ]:
!pip install themis-ml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

     |████████████████████████████████| 9.0 MB 11.9 MB/s 
  Created wheel for themis-ml: filename=themis_ml-0.0.4-py3-none-any.whl size=9829351 sha256=abe5aa364d2622ba908b320ed2900a9de249506e1ff5d667191465d31bc2ba8f
  Stored in directory: /root/.cache/pip/wheels/03/65/5e/7d4011bc7956d4385f4d3e43dda9fe3603377a8db60d11a4b8
Successfully built themis-ml
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
from themis_ml.datasets import german_credit
credit = german_credit(raw=True)
credit.head(2)
credit

,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account/bonds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,other_debtors/guarantors,...,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker,credit_risk
0,1,6,critical_account/other_credits_existing_not_at...,radio/television,1169,0,4,4,male_single,none,...,real_estate,67,none,own,2,2,1,1,1,1
1,2,48,existing_credits_paid_back_duly_till_now,radio/television,5951,1,2,2,female_divorced/separated/married,none,...,real_estate,22,none,own,1,2,1,0,1,0
2,0,12,critical_account/other_credits_existing_not_at...,education,2096,1,3,2,male_single,none,...,real_estate,49,none,own,1,1,2,0,1,1
3,1,42,existing_credits_paid_back_duly_till_now,furniture/equipment,7882,1,3,2,male_single,guarantor,...,building_society_savings_agreement/life_insurance,45,none,for free,1,2,2,0,1,1
4,1,24,delay_in_paying_off_in_the_past,car_(new),4870,1,2,3,male_single,none,...,unknown/no_property,53,none,for free,2,2,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,12,existing_credits_paid_back_duly_till_now,furniture/equipment,1736,1,3,3,female_divorced/separated/married,none,...,real_estate,31,none,own,1,1,1,0,1,1
996,1,30,existing_credits_paid_back_duly_till_now,car_(used),3857,1,2,4,male_divorced/separated,none,...,building_society_savings_agreement/life_insurance,40,none,own,1,3,1,1,1,1
997,0,12,existing_credits_paid_back_duly_till_now,radio/television,804,1,4,4,male_single,none,...,car_or_other,38,none,own,1,2,1,0,1,1
998,1,45,existing_credits_paid_back_duly_till_now,radio/television,1845,1,2,4,male_single,none,...,unknown/no_property,23,none,for free,1,2,1,1,1,0


In [ ]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                                                    Non-Null Count  Dtype 
---  ------                                                    --------------  ----- 
 0   status_of_existing_checking_account                       1000 non-null   int64 
 1   duration_in_month                                         1000 non-null   int64 
 2   credit_history                                            1000 non-null   object
 3   purpose                                                   1000 non-null   object
 4   credit_amount                                             1000 non-null   int64 
 5   savings_account/bonds                                     1000 non-null   int64 
 6   present_employment_since                                  1000 non-null   int64 
 7   installment_rate_in_percentage_of_disposable_income       1000 non-null   int64 
 8   personal_status_and_sex      

In [ ]:
credit['credit_history'].unique()

array(['critical_account/other_credits_existing_not_at_this_bank',
       'existing_credits_paid_back_duly_till_now',
       'delay_in_paying_off_in_the_past',
       'no_credits_taken/all_credits_paid_back_duly',
       'all_credits_at_this_bank_paid_back_duly'], dtype=object)

In [ ]:
credit['purpose'].unique()

array(['radio/television', 'education', 'furniture/equipment',
       'car_(new)', 'car_(used)', 'business', 'domestic_appliances',
       'repairs', 'others', 'retraining'], dtype=object)

In [ ]:
credit['housing'].unique()

array(['own', 'for free', 'rent'], dtype=object)

* Account_status: factor, status of existing checking account

* Duration: numeric, loan duration in month

* Credit_history: factor, previous credit history

* Purpose: factor, loan purpose

* Amount: numeric, credit amount

* Savings: factor, savings account/bonds

* Employment: factor, present employment since

* Installment_rate: numeric, installment rate in percentage of disposable income

* Guarantors: factor, other debtors / guarantors

* Resident_since: factor, present residence since

* Property: factor, property

* Age: numeric, age in years

* Other_plans: factor, other installment plans

* Housing: factor, housing

* Num_credits: numeric, Number of existing credits at this bank

* Job: factor, job

* People_maintenance: numeric, number of people being liable to provide maintenance for

* Phone: factor, telephone

* Foreign: factor, foreign worker

* BAD: factor, GOOD/BAD for whether a customer has defaulted on a loan. This is the outcome or target in this dataset

* Female: factor, female/male for gender

* probability: numeric, predicted probabilities for default, ranges from 0 to 1

* predicted: numeric, predicted values for default, 0/1 for no/yes

## 3.1 Literature Review

The German Credit data set is a publically available data set downloaded from the UCI Machine Learning Repository

Popular fairness metrics: 
* demographic parity
* equal opportunity
* equalized odds

Protected attributes: 

* Gender(male: privileged; female: unprivileged);

* Age(> 25: privileged; < 25 : unpreviledged)

* Nationality(non-foreigners: privileged; foreigners: unpreviledged)

## 3.2 Binary Classification Pipeline - Logistics Regression

In [ ]:
def preprocessGermanCredit(temp_df):
    df = temp_df.copy()
    
    df['credit_amount'] /= 500
    df['credit_amount'] = df['credit_amount'].astype(int)
    df.loc[df['age_in_years'] <= 25, 'age'] = 1 
    df.loc[df['age_in_years'] > 25, 'age'] = 0
    # categorical_features 
    housing = {2: 'for free', 1: 'own',  0: 'rent'}
    credit_history = {0: 'no_credits_taken/all_credits_paid_back_duly', 
                        1: 'all_credits_at_this_bank_paid_back_duly',
                        2: 'existing_credits_paid_back_duly_till_now', 
                        3: 'delay_in_paying_off_in_the_past',
                        4: 'critical_account/other_credits_existing_not_at_this_bank',
                    }
    sex = {'male_married/widowed': 1,'male_single': 1,'male_divorced/separated': 1,
           'female_divorced/separated/married': 0,
        }
    
    df["credit_history"] = df["credit_history"].map({v: k for k, v in credit_history.items()})
    df["housing"] = df["housing"].map({v: k for k,v in housing.items()})
    df["sex"] = df["personal_status_and_sex"].map({v: k for v,k in sex.items()})
    df['purpose'] = pd.factorize(df['purpose'])[0]

    df = df[['credit_risk', 'duration_in_month', 'credit_amount', 'installment_rate_in_percentage_of_disposable_income', 'age', 'credit_history', 'housing','status_of_existing_checking_account','sex', 'present_employment_since','purpose']]
    return df

In [ ]:
df = preprocessGermanCredit(credit)

In [ ]:
df['age'].value_counts()

0.0    810
1.0    190
Name: age, dtype: int64

In [ ]:
df['personal_status_and_sex'].value_counts()

1    690
0    310
Name: personal_status_and_sex, dtype: int64

In [ ]:
def logisticRegression(x_train, y_train, x_test):
    from sklearn.linear_model import LogisticRegression

    logisticRegr = LogisticRegression(solver='lbfgs', max_iter=500)
    logisticRegr.fit(x_train, y_train)

    train_predictions = logisticRegr.predict(x_train)
    test_predictions = logisticRegr.predict(x_test)
    return train_predictions, test_predictions

In [ ]:
def predictionSummary(temp_x, y_real, y_predict, df, var_name):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict, labels=[0,1]).ravel()
    # False Positive, False Negative, True Positive, True Negative, Overall Accuracy
    overall = [fp/(fp+tn),fn/(fn+tp),tp/(tp+fn),tn/(tn+fp),(tp+tn)/(tp+fp+fn+tn)]
    
    x = temp_x.copy()
    x['Predicted Creditability'] = y_predict
    x['Real Creditability'] = y_real
    x_protected = x[df[var_name] == 1].copy()
    tn, fp, fn, tp  = confusion_matrix(x_protected['Real Creditability'], x_protected['Predicted Creditability'], labels=[0,1]).ravel()
    protected = [fp/(fp+tn),fn/(fn+tp),tp/(tp+fn),tn/(tn+fp),(tp+tn)/(tp+fp+fn+tn)]

    x_unprotected = x[df[var_name] == 0].copy()
    tn, fp, fn, tp  = confusion_matrix(x_unprotected['Real Creditability'], x_unprotected['Predicted Creditability'], labels=[0,1]).ravel()
    unprotected = [fp/(fp+tn),fn/(fn+tp),tp/(tp+fn),tn/(tn+fp),(tp+tn)/(tp+fp+fn+tn)]

    return overall, unprotected, protected

In [ ]:
# Split processed data into train and test (fixed random seed)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=0)
var_name = 'sex'

In [ ]:
# Split test data into X and y
x_train, y_train = train_data.drop(["credit_risk", var_name], axis=1), train_data["credit_risk"]
x_test, y_test = test_data.drop(["credit_risk", var_name], axis=1), test_data["credit_risk"]

# Run the binary classification pipeline
y_train_predict, y_test_predict = logisticRegression(x_train, y_train, x_test)
# Save results for each i and for each epsilon
train_overall, train_unprotected, train_protected = predictionSummary(x_train, y_train, y_train_predict, train_data, var_name)
test_overall, test_unprotected, test_protected = predictionSummary(x_test, y_test, y_test_predict, test_data, var_name)

In [ ]:
# Gender Summary
print("Training Accuracy:", train_overall[-1])
print("Testing Accuracy:", test_overall[-1])
print("Training TPR diff", train_protected[2]-train_unprotected[2])
print("Testing TPR diff", test_protected[2]-test_unprotected[2])
print("Training FPR diff", train_protected[0]-train_unprotected[0])
print("Testing FPR diff", test_protected[0]-test_unprotected[0])

Training Accuracy: 0.735
Testing Accuracy: 0.695
Training TPR diff -0.026649093480776598
Testing TPR diff -0.040985442329227295
Training FPR diff 0.07790837854128996
Testing FPR diff -0.06303030303030299


In [ ]:
var_name = 'age'

In [ ]:
# Split test data into X and y
x_train, y_train = train_data.drop(["credit_risk", var_name], axis=1), train_data["credit_risk"]
x_test, y_test = test_data.drop(["credit_risk", var_name], axis=1), test_data["credit_risk"]

# Run the binary classification pipeline
y_train_predict, y_test_predict = logisticRegression(x_train, y_train, x_test)
# Save results for each i and for each epsilon
train_overall, train_unprotected, train_protected = predictionSummary(x_train, y_train, y_train_predict, train_data, var_name)
test_overall, test_unprotected, test_protected = predictionSummary(x_test, y_test, y_test_predict, test_data, var_name)

In [ ]:
# Age Summary
print("Training Accuracy:", train_overall[-1])
print("Testing Accuracy:", test_overall[-1])
print("Training TPR diff", train_protected[2]-train_unprotected[2])
print("Testing TPR diff", test_protected[2]-test_unprotected[2])
print("Training FPR diff", train_protected[0]-train_unprotected[0])
print("Testing FPR diff", test_protected[0]-test_unprotected[0])

Training Accuracy: 0.73375
Testing Accuracy: 0.685
Training TPR diff -0.05641025641025643
Testing TPR diff 0.1057377049180327
Training FPR diff -0.07096774193548383
Testing FPR diff 0.05277777777777781


## 3.3 Hypothesis Test Pipeline

In [ ]:
from scipy.stats import ttest_ind
def hypothesisTest(credit):

    # H0: null hypothesis: the mean of the credit risk of people with age lower then 25 = the mean of the  credit risk of people with age high then 25
    # H1: alternate hypothesis: the mean of the credit risk of people with age lower then 25 > the mean of the  credit risk of people with age high then 25
    
    young = credit[credit['age_in_years'] <25].copy()
    old = credit[credit['age_in_years'] >=25].copy()
    young_credit_risk =  young['credit_risk']
    old_credit_risk = old['credit_risk']

    stat, p = ttest_ind(old_credit_risk, young_credit_risk)
    print('first:')
    print('stat=%.3f, p=%.27f' % (stat, p))

    if p > 0.05:
        print('we are accepting null hypothesis, Probably the same distribution')
    else:
        print('we are rejecting null hypothesis, Probably different distributions')
    
    # so the result shows us the the mean rate are not the same 
    # Tests whether the means of two independent samples are significantly different.







    # H0: null hypothesis: the mean of the credit risk of people with age lower then 25 = the mean of the  credit risk of people with age high then 25
    # H1: alternate hypothesis: the mean of the credit risk of people with age lower then 25 > the mean of the  credit risk of people with age high then 25
    
    x_male = credit[(credit['personal_status_and_sex'] == 'male_single ')].copy()
        # (credit['personal_status_and_sex'] == 'male_married/widowed')| 
        #        (credit['personal_status_and_sex'] == 'male_single ') |
        #        (credit['personal_status_and_sex'] == 'male_divorced/separated')].copy()
    x_female = credit[credit['personal_status_and_sex'] == 'female_divorced/separated/married'].copy()

    female_credit_risk =  x_female['credit_risk']
    male_credit_risk = x_male['credit_risk']

    stat, p = ttest_ind(female_credit_risk, male_credit_risk)
    print('first:')
    print('stat=%.3f, p=%.27f' % (stat, p))

    if p > 0.05:
        print('we are accepting null hypothesis, Probably the same distribution')
    else:
        print('we are rejecting null hypothesis, Probably different distributions')
    
    # so the result shows us the the mean rate are not the same 
    # Tests whether the means of two independent samples are significantly different.








    # H0: null hypothesis: the mean of the credit risk of foreign worker = the mean of the  credit risk of non-foreigner
    # H1: alternate hypothesis: the mean of the credit risk of foreign worker < the mean of the  credit risk of non-foreigner

    
    non_foreign  = credit[credit['foreign_worker'] == 1].copy()

    foreigner = credit[credit['foreign_worker'] == 0].copy()

    foreign_credit_risk =  foreigner['credit_risk']
    non_foreigner_credit_risk = non_foreign['credit_risk']

    stat, p = ttest_ind(foreign_credit_risk, non_foreigner_credit_risk)
    print('first:')
    print('stat=%.3f, p=%.27f' % (stat, p))

    if p > 0.05:
        print('we are accepting null hypothesis, Probably the same distribution')
    else:
        print('we are rejecting null hypothesis, Probably different distributions')
    
    # so the result shows us the the mean rate are not the same 
    # Tests whether the means of two independent samples are significantly different.

hypothesisTest(credit)


first:
stat=3.171, p=0.001562879701362014997489958
we are rejecting null hypothesis, Probably different distributions
first:
stat=nan, p=nan
we are rejecting null hypothesis, Probably different distributions
first:
stat=2.602, p=0.009411920075301638891063227
we are rejecting null hypothesis, Probably different distributions


The above Hypothesis Testing corresponding with the following protected attributes:

Gender(male: privileged; female: unprivileged);

Age(> 25: privileged; < 25 : unpreviledged)

Nationality(non-foreigners: privileged; foreigners: unpreviledged)

## 3.4 References

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=aa928c63-0856-4021-a156-02d09e35182a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>